In [45]:
# The code was removed by DSX for sharing.

<h1>Yahoo SEM Click Data</h1>
<h2>
The tab separated data has the following fields:
    <ul>
    <li>0 day </li>
    <li>1 anonymized account_id </li>
    <li>2 rank </li>
    <li>3 anonymized keyphrase (list of anonymized keywords) </li>
    <li>4 avg bid </li>
    <li>5 impressions </li>
    <li>6 clicks </li>
    </ul>
</h2>

In [46]:
sc.textFile(path_1).first()

u'99\t9f9893bd-5eb5-4e79-9e85-48f3b6e33375\t4\t3db691494440189b dff40de4dd25253a 12f7d2408c3dcb88\t540.0\t218.0\t0.0'

In [36]:
import math
rdd = sc.textFile(path_1).map(lambda line: (line.split("\t")[3],(float(line.split("\t")[-2]),float(line.split("\t")[-1]))))
rdd.first()

(u'3db691494440189b dff40de4dd25253a 12f7d2408c3dcb88', (218.0, 0.0))

In [37]:
rdd = rdd.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).mapValues(lambda x: 1 if (x[1]/x[0])>0 else 0)

In [38]:
keyphrases0 = rdd.filter(lambda x: x[1]==0)
keyphrases1 = rdd.filter(lambda x: x[1]==1)

In [39]:
rdd0 = keyphrases0.flatMap(lambda x: [(e,1) for e in x[0].split()]) 
rdd1 = keyphrases1.flatMap(lambda x: [(e,1) for e in x[0].split()])

In [40]:
X = keyphrases1.count() 
Y = keyphrases0.count()
print X,Y

2550 31153


In [41]:
t_rdd0 = rdd0.reduceByKey(lambda x,y: x+y).mapValues(lambda y: (y+0.5)/(Y-y+0.5))
t_rdd1 = rdd1.reduceByKey(lambda x,y: x+y).mapValues(lambda x: (x+0.5)/(X-x+0.5))

In [42]:
ct_rdd = t_rdd1.join(t_rdd0).mapValues(lambda r: math.log(r[0] / r[1]))

In [43]:
cts = sc.broadcast(dict(ct_rdd.collect()))

In [44]:
def accuracy(rdd, cts, threshold):
    csv_rdd = rdd.map(lambda x: (x[0], x[1], sum([cts.value[t] for t in x[0].split() if t in cts.value])))
    results = csv_rdd.map(lambda x: (x[1] == (1 if x[2] > threshold else 0),1)).reduceByKey(lambda x,y: x+y).collect()
    print float(results[1][1]) / (results[0][1]+results[1][1])

accuracy(rdd, cts, 10)
accuracy(rdd, cts, -10)

0.924339079607
0.0756609203928
